# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline
from sklearn import preprocessing

In [2]:
def normalize_col(col):
    col = col.values.reshape(-1, 1).astype("float64")
    min_max_scaler = preprocessing.MinMaxScaler()
    scaled_col = min_max_scaler.fit_transform(col)
    return scaled_col

In [3]:
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [3]:
def process_portfolio(filepath="data/portfolio.json"):
    # read in the json files
    portfolio = pd.read_json(filepath, orient='records', lines=True)

    # Step1: make columns for different offer
    portfolio = pd.concat([
        portfolio,
        pd.get_dummies(portfolio["offer_type"], prefix="offer_type")
    ], axis=1).drop(["offer_type"], axis=1)
    
    # Step2: for each channel-type make a new column
    portfolio["channel_mobile"] = portfolio["channels"].apply(lambda x: int("mobile" in x))
    portfolio["channel_web"] = portfolio["channels"].apply(lambda x: int("web" in x))
    portfolio["channel_social"] = portfolio["channels"].apply(lambda x: int("social" in x))
    portfolio["channel_email"] = portfolio["channels"].apply(lambda x: int("email" in x))
    
    # Step3: create new two ratio features
    portfolio["difficulty_duration"] = portfolio["difficulty"] / portfolio["duration"]
    portfolio["reward_difficulty"] = portfolio["reward"] / portfolio["difficulty"]
    portfolio["reward_difficulty"] = portfolio["reward_difficulty"].fillna(0)
    
    # Final step: normalize columns
    portfolio["reward"] = normalize_col(portfolio["reward"])
    portfolio["difficulty"] = normalize_col(portfolio["difficulty"])
    portfolio["duration"] = normalize_col(portfolio["duration"])    
    
    # drop channels
    return portfolio.drop(["channels"], axis=1)

In [4]:
# read in the json files
portfolio_df = process_portfolio()
portfolio_df.head()
# profile = pd.read_json('data/profile.json', orient='records', lines=True)
# transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

,reward,difficulty,duration,id,offer_type_bogo,offer_type_discount,offer_type_informational,channel_mobile,channel_web,channel_social,channel_email,difficulty_duration,reward_difficulty
0,1.0,0.50,0.571429,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,1,0,1,1,1.428571,1.00
1,1.0,0.50,0.285714,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1,1,2.000000,1.00
2,0.0,0.00,0.142857,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,0,1,0.000000,0.00
3,0.5,0.25,0.571429,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,0,1,0.714286,1.00
4,0.5,1.00,1.000000,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,0,1,0,1,2.000000,0.25


In [8]:
profile_df = pd.read_json('data/profile.json', orient='records', lines=True)
profile_df.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [5]:
def process_profile(filepath='data/profile.json'):
    # read in the json files
    profile = pd.read_json(filepath, orient='records', lines=True)
    
    # Step1: encoding for gender
    profile = pd.concat([profile, 
                         pd.get_dummies(profile['gender'], prefix='gender', dummy_na=True)
                        ], axis=1).drop(["gender"], axis=1)
    
    # Step2: fillin nan income with median
    profile = profile.fillna(
        {"income": profile.income.dropna().median()})
    
    # Step3: fillin nan age with 118
    profile.age = profile.age.apply(lambda x: None if x == 118 else x)
    profile = profile.fillna({"age": profile.age.dropna().median()})
    
    # Step4: convert date to unix timestamp
    profile.became_member_on = pd.to_datetime(
        profile.became_member_on, format='%Y%m%d').astype(np.int64) // 10**9
    
    # normalize
    profile["income"] = normalize_col(profile["income"])
    profile["age"] = normalize_col(profile["age"])
    profile["became_member_on"] = normalize_col(profile["became_member_on"])

    
    return profile

In [9]:
profile_df = process_profile()
profile_df.head()

,age,id,became_member_on,income,gender_F,gender_M,gender_O,gender_nan
0,0.445783,68be06ca386d4c31939f3a4f0e3dd783,0.709819,0.377778,0,0,0,1
1,0.445783,0610b486422d4921ae7d2bf64640c50b,0.793747,0.911111,1,0,0,0
2,0.445783,38fe809add3b4fcf9315a9694bb96ff5,0.992320,0.377778,0,0,0,1
3,0.686747,78afa995795e4d85b5d9ceeca43f5fef,0.756994,0.777778,1,0,0,0
4,0.445783,a03223e636434f42ac4c3df47e8bac43,0.804717,0.377778,0,0,0,1


In [7]:
transcript_df = pd.read_csv('data/processed_transcript.csv')
transcript_df.head()

,person,offer_id,offer_type,difficulty,amount,receive_time,view_time,complete_time,expected_complete_time,is_in_expected_complete_time,is_enough_amount,is_view_event,is_complete_event,is_complete
0,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,informational,0.0,22.16,168,192.0,NaN,240.0,False,True,True,False,True
1,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,informational,0.0,8.57,336,372.0,NaN,432.0,False,True,True,False,True
2,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5.0,8.57,408,456.0,414.0,528.0,True,True,True,True,True
3,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,discount,10.0,14.11,504,540.0,528.0,744.0,True,True,True,True,True
4,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,discount,10.0,10.27,576,NaN,576.0,744.0,True,True,False,True,False


In [5]:
print(transcript_df.shape)

(76277, 14)


In [10]:
# merge three tables
df = transcript_df[["person", "offer_id", "is_complete"]].\
merge(profile_df, left_on="person", right_on="id").drop(["id", "person"], axis=1).\
merge(portfolio_df, left_on="offer_id", right_on="id").drop(["id", "offer_id"], axis=1)

In [11]:
# make is_complete 1 or 0
df.is_complete = df.is_complete.astype(int)

In [12]:
df.shape

(76277, 20)

In [15]:
df.head()

,is_complete,age,became_member_on,income,gender_F,gender_M,gender_O,gender_nan,reward,difficulty,duration,offer_type_bogo,offer_type_discount,offer_type_informational,channel_mobile,channel_web,channel_social,channel_email,difficulty_duration,reward_difficulty
0,1,0.180723,0.747120,0.466667,0,1,0,0,0.0,0.0,0.0,0,0,1,1,0,1,1,0.0,0.0
1,0,0.265060,0.891388,0.300000,0,0,1,0,0.0,0.0,0.0,0,0,1,1,0,1,1,0.0,0.0
2,1,0.493976,0.520570,0.666667,1,0,0,0,0.0,0.0,0.0,0,0,1,1,0,1,1,0.0,0.0
3,1,0.072289,0.658804,0.333333,1,0,0,0,0.0,0.0,0.0,0,0,1,1,0,1,1,0.0,0.0
4,1,0.096386,0.780581,0.477778,1,0,0,0,0.0,0.0,0.0,0,0,1,1,0,1,1,0.0,0.0


In [13]:
# split into train/test
def train_test_split(df, train_frac= 0.7, seed=666):
    '''Randomly shuffle the data and split it into training and test features and labels using the train_frac
       :return: Two tuples (in order): (train_features, train_labels), (test_features, test_labels)
       '''
    
    # shuffle and break the data
    df_matrix = df.values
    
    np.random.seed(seed)
    np.random.shuffle(df_matrix)
    
    train_size = int(df_matrix.shape[0] * train_frac)
    train_features = df_matrix[:train_size, 1:]
    train_labels = df_matrix[:train_size, 0]
    
    test_features = df_matrix[train_size:, 1:]
    test_labels = df_matrix[train_size:, 0]
    
    return (train_features, train_labels), (test_features, test_labels)


In [14]:
(train_x, train_y), (test_x, test_y) = train_test_split(df)

In [15]:
print(train_x.shape, train_y.shape)

(53393, 19) (53393,)


In [16]:
print(test_x.shape, test_y.shape)

(22884, 19) (22884,)


## Uploading the Data to S3 and making a local copy of the data

In [28]:
pd.concat([
    pd.DataFrame(train_y), 
    pd.DataFrame(train_x)
], axis=1).to_csv("data/train.csv", header=False, index=False)

In [29]:
pd.concat([
    pd.DataFrame(test_y), 
    pd.DataFrame(test_x)
], axis=1).to_csv("data/test.csv", header=False, index=False)

In [30]:
pd.DataFrame(test_x).to_csv("data/test_x.csv", header=False, index=False)

In [31]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

session = sagemaker.Session()
role = get_execution_role()
prefix = 'starbucks-xgboost'


In [32]:
train_location = session.upload_data("data/train.csv", key_prefix=prefix)
train_location

's3://sagemaker-us-east-2-290062341908/starbucks-xgboost/train.csv'

In [33]:
test_location = session.upload_data("data/test_x.csv", key_prefix=prefix)
test_location

's3://sagemaker-us-east-2-290062341908/starbucks-xgboost/test_x.csv'

## Train the XGBoost model

In [39]:
container = get_image_uri(session.boto_region_name, 'xgboost')

xgb = sagemaker.estimator.Estimator(
    container, # The name of the training container
    role,      # The IAM role to use (our current role in this case)
    train_instance_count=1, # The number of instances to use for training
    train_instance_type='ml.m4.xlarge', # The type of instance ot use for training
    output_path=f's3://{session.default_bucket()}/{prefix}/output',
                                        # Where to save the output (the model artifacts)
    sagemaker_session=session) # The current SageMaker session


# hyperparameters
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        objective='binary:logistic',
                        early_stopping_rounds=10,
                        num_round=200)


s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='csv')

xgb.fit({'train': s3_input_train})

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
There is a more up to date SageMaker XGBoost image. To use the newer image, please set 'repo_version'='1.0-1'. For example:
	get_image_uri(region, 'xgboost', '1.0-1').
Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-08-01 23:11:34 Starting - Starting the training job...
2020-08-01 23:11:36 Starting - Launching requested ML instances......
2020-08-01 23:12:38 Starting - Preparing the instances for training...
2020-08-01 23:13:28 Downloading - Downloading input data...
2020-08-01 23:14:02 Training - Training image download completed. Training in progress..Arguments: train
[2020-08-01:23:14:03:INFO] Running standalone xgboost training.
[2020-08-01:23:14:03:INFO] Path /opt/ml/input/data/validation does not exist!
[2020-08-01:23:14:03:INFO] File size need to be processed in the node: 7.56mb. Available memory size in the node: 8491.77mb
[2020-08-01:23:14:03:INFO] Determined delimiter of CSV input is ','
[23:14:03] S3DistributionType set as FullyReplicated
[23:14:03] 53393x19 matrix with 1014467 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[23:14:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 4 pruned nodes, max_depth=5
[0]#011train


2020-08-01 23:14:19 Uploading - Uploading generated training model
2020-08-01 23:14:19 Completed - Training job completed
Training seconds: 51
Billable seconds: 51


In [40]:
xgb_transformer = xgb.transformer(instance_count = 1, instance_type = 'ml.m4.xlarge')
xgb_transformer.transform(test_location, content_type='text/csv', split_type='Line')
xgb_transformer.wait()

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


....................Arguments: serve
[2020-08-01 23:17:59 +0000] [1] [INFO] Starting gunicorn 19.7.1
[2020-08-01 23:17:59 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2020-08-01 23:17:59 +0000] [1] [INFO] Using worker: gevent
[2020-08-01 23:17:59 +0000] [37] [INFO] Booting worker with pid: 37
[2020-08-01 23:17:59 +0000] [38] [INFO] Booting worker with pid: 38
[2020-08-01 23:17:59 +0000] [39] [INFO] Booting worker with pid: 39
[2020-08-01 23:17:59 +0000] [40] [INFO] Booting worker with pid: 40
[2020-08-01:23:17:59:INFO] Model loaded successfully for worker : 37
[2020-08-01:23:17:59:INFO] Model loaded successfully for worker : 38
[2020-08-01:23:17:59:INFO] Model loaded successfully for worker : 40
[2020-08-01:23:17:59:INFO] Model loaded successfully for worker : 39

[2020-08-01:23:18:07:INFO] Sniff delimiter as ','
[2020-08-01:23:18:07:INFO] Determined delimiter of CSV input is ','
[2020-08-01:23:18:07:INFO] Sniff delimiter as ','
[2020-08-01:23:18:07:INFO] Determined delimit

In [41]:
!aws s3 cp --recursive $xgb_transformer.output_path $"data"

download: s3://sagemaker-us-east-2-290062341908/xgboost-2020-08-01-23-14-46-219/test_x.csv.out to data/test_x.csv.out


In [43]:
predictions = pd.read_csv("data/test_x.csv.out", header=None)

In [44]:
predictions = [round(num) for num in predictions.squeeze().values]

In [45]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.7213773815766474

In [46]:
from sklearn.metrics import f1_score
f1_score(test_y, predictions)

0.6820584422060436

In [47]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, predictions)

0.7160378193928134

## Training PyTorch Neural Network Model

In [26]:
import pandas as pd
import boto3
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

bucket = sagemaker_session.default_bucket()
prefix = "starbucks-pytorch"

In [54]:
# using PyTorch
from sagemaker.pytorch import PyTorch

output_path = f"s3://{bucket}/{prefix}"

# instantiate a pytorch estimator
estimator = PyTorch(entry_point="train.py", source_dir="pytorch",
    role=role, framework_version="1.0", train_instance_count=1,
    train_instance_type="ml.p2.xlarge", output_path=output_path,
    sagemaker_session=sagemaker_session, 
    hyperparameters={
        "input_features": 19,
        "hidden_dim": 30, 
        "output_dim": 1,
        "epochs": 100
    })


In [58]:
estimator.fit({'train': s3_input_train})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-08-01 23:32:59 Starting - Starting the training job...
2020-08-01 23:33:01 Starting - Launching requested ML instances......
2020-08-01 23:34:03 Starting - Preparing the instances for training.........
2020-08-01 23:35:53 Downloading - Downloading input data
2020-08-01 23:35:53 Training - Downloading the training image......
2020-08-01 23:36:52 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-08-01 23:36:53,876 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-08-01 23:36:53,902 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-08-01 23:36:56,919 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-08-01 23:36:57,157 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-08-01 23:36:57,157 sagem

Epoch: 40, Loss: 0.5631655686915144
Epoch: 41, Loss: 0.5640585910124278
Epoch: 42, Loss: 0.5634665642115061
Epoch: 43, Loss: 0.5641576561058282
Epoch: 44, Loss: 0.5647241723169102
Epoch: 45, Loss: 0.5642444735218524
Epoch: 46, Loss: 0.564201508747058
Epoch: 47, Loss: 0.5630445745386434
Epoch: 48, Loss: 0.5628977005819926
Epoch: 49, Loss: 0.5636496361554338
Epoch: 50, Loss: 0.563911111465889
Epoch: 51, Loss: 0.5636467011875651
Epoch: 52, Loss: 0.5637597518941659
Epoch: 53, Loss: 0.564462282115154
Epoch: 54, Loss: 0.5628810370766492
Epoch: 55, Loss: 0.5635869333220556
Epoch: 56, Loss: 0.5632949665049265
Epoch: 57, Loss: 0.5626511592404003
Epoch: 58, Loss: 0.5637366153104475
Epoch: 59, Loss: 0.563914757104737
Epoch: 60, Loss: 0.5633158057286275
Epoch: 61, Loss: 0.5631143637308467
Epoch: 62, Loss: 0.5633649985776858
Epoch: 63, Loss: 0.5636755887963129
Epoch: 64, Loss: 0.563419999687301
Epoch: 65, Loss: 0.5632121113160353
Epoch: 66, Loss: 0.5641047578430578
Epoch: 67, Loss: 0.56393589001246

In [59]:
# Deploy the trained PyTorch model
from sagemaker.pytorch import PyTorchModel

model = PyTorchModel(
    entry_point="predict.py",
    role=role, 
    framework_version="1.0",
    model_data=estimator.model_data,
    source_dir="pytorch"
)

# deploy your model to create a predictor
predictor = model.deploy(initial_instance_count=1, instance_type="ml.t2.medium")


Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


-------------!

In [60]:
# evaluating the model
# read in test data, assuming it is stored locally
test_data = pd.read_csv("data/test_full.csv", header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]
test_y_preds = np.hstack(
    predictor.predict(test_x.iloc[i: i+1000]).reshape(1, -1).squeeze() 
    for i in range(0, len(test_x), 1000))

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:10: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


In [61]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.7213773815766474

In [62]:
# calculate the test roc_auc_score
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, np.hstack([i.reshape(1, -1).squeeze()] for i in test_y_preds).squeeze())

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  app.launch_new_instance()


0.6766610485537375

In [63]:
predictor.delete_endpoint()

## Hyperparameter tuning

## lower hidden dim

In [65]:
# specify an output path
output_path = f"s3://{bucket}/{prefix}-lower-hidden-dim"

# instantiate a pytorch estimator
estimator = PyTorch(
    entry_point="train.py",
    source_dir="pytorch",
    role=role,
    framework_version="1.0",
    train_instance_count=1,
    train_instance_type="ml.p2.xlarge", # "ml.c4.xlarge",
    output_path=output_path,
    sagemaker_session=sagemaker_session,
    hyperparameters={
        "input_features": 19,
        "hidden_dim": 15, 
        "output_dim": 1,
        "epochs": 100
    })
estimator.fit({'train': s3_input_train})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-08-02 04:23:27 Starting - Starting the training job...
2020-08-02 04:23:29 Starting - Launching requested ML instances......
2020-08-02 04:24:31 Starting - Preparing the instances for training......
2020-08-02 04:25:56 Downloading - Downloading input data...
2020-08-02 04:26:20 Training - Downloading the training image......
2020-08-02 04:27:16 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-08-02 04:27:17,382 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-08-02 04:27:17,411 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-08-02 04:27:18,870 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-08-02 04:27:19,112 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-08-02 04:27:19,112 sagem

Epoch: 40, Loss: 0.5691882857418016
Epoch: 41, Loss: 0.5697584757699948
Epoch: 42, Loss: 0.5689942342204771
Epoch: 43, Loss: 0.5702198976434572
Epoch: 44, Loss: 0.5688601532651021
Epoch: 45, Loss: 0.5697743418204427
Epoch: 46, Loss: 0.5707387903461072
Epoch: 47, Loss: 0.5702929901914874
Epoch: 48, Loss: 0.5697801967182856
Epoch: 49, Loss: 0.5690218874577726
Epoch: 50, Loss: 0.5699533915279733
Epoch: 51, Loss: 0.5698173541868671
Epoch: 52, Loss: 0.5696064603351029
Epoch: 53, Loss: 0.5690010259018632
Epoch: 54, Loss: 0.5700222345885266
Epoch: 55, Loss: 0.5692665423793293
Epoch: 56, Loss: 0.5698742443200354
Epoch: 57, Loss: 0.5693693534321106
Epoch: 58, Loss: 0.5701889367976438
Epoch: 59, Loss: 0.5700939099133461
Epoch: 60, Loss: 0.569752828146188
Epoch: 61, Loss: 0.5697249274575309
Epoch: 62, Loss: 0.5694402349547724
Epoch: 63, Loss: 0.568466367176298
Epoch: 64, Loss: 0.5693105741135414
Epoch: 65, Loss: 0.5699430717511124
Epoch: 66, Loss: 0.5693739241344875
Epoch: 67, Loss: 0.56949883712

In [66]:
# Deploy the trained model
from sagemaker.pytorch import PyTorchModel
from sklearn.metrics import roc_auc_score

model = PyTorchModel(
    entry_point="predict.py",
    role=role, 
    framework_version="1.0",
    model_data=estimator.model_data,
    source_dir="pytorch"
)

# deploy your model to create a predictor
predictor = model.deploy(initial_instance_count=1, instance_type="ml.t2.medium")

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


---------------!

In [67]:
# evaluating the model
# read in test data, assuming it is stored locally
test_data = pd.read_csv("data/test_full.csv", header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]
test_y_preds = np.hstack(
    predictor.predict(test_x.iloc[i: i+1000]).reshape(1, -1).squeeze() 
    for i in range(0, len(test_x), 1000))

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:10: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


In [68]:
# calculate the test roc_auc_score
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, test_y_preds)

0.6783056209109046

In [69]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, test_y_preds)

0.6709928334207307

In [70]:
from sklearn.metrics import f1_score
f1_score(test_y, test_y_preds)

0.6795488401787614

In [ ]:
predictor.delete_endpoint()

## Higher epochs

In [71]:
# import a PyTorch wrapper
from sagemaker.pytorch import PyTorch

# specify an output path
output_path = f"s3://{bucket}/{prefix}-higher-epochs"

# instantiate a pytorch estimator
estimator = PyTorch(
    entry_point="train.py",
    source_dir="pytorch",
    role=role,
    framework_version="1.0",
    train_instance_count=1,
    train_instance_type="ml.c4.xlarge",
    output_path=output_path,
    sagemaker_session=sagemaker_session,
    hyperparameters={
        "input_features": 19,
        "hidden_dim": 15, 
        "output_dim": 1,
        "epochs": 200
    })
estimator.fit({'train': s3_input_train})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-08-02 05:21:19 Starting - Starting the training job...
2020-08-02 05:21:21 Starting - Launching requested ML instances......
2020-08-02 05:22:25 Starting - Preparing the instances for training...
2020-08-02 05:23:11 Downloading - Downloading input data...
2020-08-02 05:23:47 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-08-02 05:23:48,771 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-08-02 05:23:48,774 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-08-02 05:23:48,786 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-08-02 05:23:50,210 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-08-02 05:23:50,451 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.p

Epoch: 39, Loss: 0.5710417843350534
Epoch: 40, Loss: 0.570806183155333
Epoch: 41, Loss: 0.5706702093450764
Epoch: 42, Loss: 0.5696929525179363
Epoch: 43, Loss: 0.5719858081934603
Epoch: 44, Loss: 0.5699253421597713
Epoch: 45, Loss: 0.5705500151278374
Epoch: 46, Loss: 0.5703532095454382
Epoch: 47, Loss: 0.5706414050405169
Epoch: 48, Loss: 0.5696726559727603
Epoch: 49, Loss: 0.569334594280309
Epoch: 50, Loss: 0.5703028764980339
Epoch: 51, Loss: 0.5703907062972753
Epoch: 52, Loss: 0.5698256547810433
Epoch: 53, Loss: 0.5708259495903044
Epoch: 54, Loss: 0.5704331443746214
Epoch: 55, Loss: 0.56993686201309
Epoch: 56, Loss: 0.5719337852567099
Epoch: 57, Loss: 0.5685718259170707
Epoch: 58, Loss: 0.5704483584267147
Epoch: 59, Loss: 0.5708758233247625
Epoch: 60, Loss: 0.5703394589966603
Epoch: 61, Loss: 0.5692874966628766
Epoch: 62, Loss: 0.5692042834470781
Epoch: 63, Loss: 0.5712606833054779
Epoch: 64, Loss: 0.5703451147993629
Epoch: 65, Loss: 0.5705255122750663
Epoch: 66, Loss: 0.5693927068444

In [72]:
# Deploy the trained model
from sagemaker.pytorch import PyTorchModel
from sklearn.metrics import roc_auc_score

model = PyTorchModel(
    entry_point="predict.py",
    role=role, 
    framework_version="1.0",
    model_data=estimator.model_data,
    source_dir="pytorch"
)

# deploy your model to create a predictor
predictor = model.deploy(initial_instance_count=1, instance_type="ml.t2.medium")

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


---------------!

In [73]:
# evaluating the model
# read in test data, assuming it is stored locally
test_data = pd.read_csv("data/test_full.csv", header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]
test_y_preds = np.hstack(
    predictor.predict(test_x.iloc[i: i+1000]).reshape(1, -1).squeeze() 
    for i in range(0, len(test_x), 1000))

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:10: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


In [74]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, test_y_preds)

0.6748383149798987

In [75]:
from sklearn.metrics import f1_score
f1_score(test_y, test_y_preds)

0.6808218590485996

In [76]:
# calculate the test roc_auc_score
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, test_y_preds)

0.6814906957161151

In [93]:
predictor.delete_endpoint()

ClientError: An error occurred (ValidationException) when calling the DeleteEndpointConfig operation: Could not find endpoint configuration "arn:aws:sagemaker:us-east-2:290062341908:endpoint-config/sagemaker-pytorch-2020-08-02-06-41-21-237".

## 6x hidden layers

In [94]:
# import a PyTorch wrapper
from sagemaker.pytorch import PyTorch

# specify an output path
output_path = f"s3://{bucket}/{prefix}-5-hidden-layer"

# instantiate a pytorch estimator
estimator = PyTorch(
    entry_point="train.py",
    source_dir="pytorch",
    role=role,
    framework_version="1.0",
    train_instance_count=1,
    train_instance_type="ml.c4.xlarge",
    output_path=output_path,
    sagemaker_session=sagemaker_session,
    hyperparameters={
        "input_features": 19,
        "hidden_dim": 100, 
        "output_dim": 1,
        "epochs": 100
    })
estimator.fit({'train': s3_input_train})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-08-02 06:57:15 Starting - Starting the training job...
2020-08-02 06:57:17 Starting - Launching requested ML instances.........
2020-08-02 06:58:50 Starting - Preparing the instances for training...
2020-08-02 06:59:42 Downloading - Downloading input data
2020-08-02 06:59:42 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-08-02 06:59:58,381 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-08-02 06:59:58,384 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-08-02 06:59:58,396 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-08-02 07:00:01,421 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-08-02 07:00:01,734 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-08-02 07:00:01,734

Epoch: 35, Loss: 0.5565894423910741
Epoch: 36, Loss: 0.5569022802991813
Epoch: 37, Loss: 0.5560901731671242
Epoch: 38, Loss: 0.5570221446231287
Epoch: 39, Loss: 0.5564758293637622
Epoch: 40, Loss: 0.5565909307491913
Epoch: 41, Loss: 0.5567041448886029
Epoch: 42, Loss: 0.5557181562684225
Epoch: 43, Loss: 0.5563097157281883
Epoch: 44, Loss: 0.5561630282341764
Epoch: 45, Loss: 0.556248273173075
Epoch: 46, Loss: 0.556177474893211
Epoch: 47, Loss: 0.5558486906972048
Epoch: 48, Loss: 0.5561224925858474
Epoch: 49, Loss: 0.5560330238737418
Epoch: 50, Loss: 0.5549440956768695
Epoch: 51, Loss: 0.5556530242695121
Epoch: 52, Loss: 0.5561214016896955
Epoch: 53, Loss: 0.55566878280278
Epoch: 54, Loss: 0.5556357022835298
Epoch: 55, Loss: 0.5555755676206354
Epoch: 56, Loss: 0.5554641690230772
Epoch: 57, Loss: 0.5555509729610847
Epoch: 58, Loss: 0.5559595439792349
Epoch: 59, Loss: 0.554919367233354
Epoch: 60, Loss: 0.5557441146493182
Epoch: 61, Loss: 0.5560518335285928
Epoch: 62, Loss: 0.55583821348706

In [95]:
# Deploy the trained model
from sagemaker.pytorch import PyTorchModel
from sklearn.metrics import roc_auc_score

model = PyTorchModel(
    entry_point="predict.py",
    role=role, 
    framework_version="1.0",
    model_data=estimator.model_data,
    source_dir="pytorch"
)

# deploy your model to create a predictor
predictor = model.deploy(initial_instance_count=1, instance_type="ml.t2.medium")

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


---------------!

In [96]:
# evaluating the model
# read in test data, assuming it is stored locally
test_data = pd.read_csv("data/test_full.csv", header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]
test_y_preds = np.hstack(
    predictor.predict(test_x.iloc[i: i+1000]).reshape(1, -1).squeeze() 
    for i in range(0, len(test_x), 1000))

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:10: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


In [97]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, test_y_preds)

0.6580580318126201

In [98]:
from sklearn.metrics import f1_score
f1_score(test_y, test_y_preds)

0.6716461751500147

In [99]:
# calculate the test roc_auc_score
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, test_y_preds)

0.6665465522617937

In [100]:
predictor.delete_endpoint()

## no dropout

In [101]:
# import a PyTorch wrapper
from sagemaker.pytorch import PyTorch

# specify an output path
output_path = f"s3://{bucket}/{prefix}-5-hidden-layer"

# instantiate a pytorch estimator
estimator = PyTorch(
    entry_point="train.py",
    source_dir="pytorch",
    role=role,
    framework_version="1.0",
    train_instance_count=1,
    train_instance_type="ml.c4.xlarge",
    output_path=output_path,
    sagemaker_session=sagemaker_session,
    hyperparameters={
        "input_features": 19,
        "hidden_dim": 100, 
        "output_dim": 1,
        "epochs": 100
    })
estimator.fit({'train': s3_input_train})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-08-02 07:26:06 Starting - Starting the training job...
2020-08-02 07:26:07 Starting - Launching requested ML instances......
2020-08-02 07:27:36 Starting - Preparing the instances for training.........
2020-08-02 07:28:58 Downloading - Downloading input data
2020-08-02 07:28:58 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-08-02 07:29:14,695 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-08-02 07:29:14,698 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-08-02 07:29:14,711 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-08-02 07:29:14,712 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-08-02 07:29:14,961 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-08-02 07:29:14,

Epoch: 35, Loss: 0.5565894423910741
Epoch: 36, Loss: 0.5569022802991813
Epoch: 37, Loss: 0.5560901731671242
Epoch: 38, Loss: 0.5570221446231287
Epoch: 39, Loss: 0.5564758293637622
Epoch: 40, Loss: 0.5565909307491913
Epoch: 41, Loss: 0.5567041448886029
Epoch: 42, Loss: 0.5557181562684225
Epoch: 43, Loss: 0.5563097157281883
Epoch: 44, Loss: 0.5561630282341764
Epoch: 45, Loss: 0.556248273173075
Epoch: 46, Loss: 0.556177474893211
Epoch: 47, Loss: 0.5558486906972048
Epoch: 48, Loss: 0.5561224925858474
Epoch: 49, Loss: 0.5560330238737418
Epoch: 50, Loss: 0.5549440956768695
Epoch: 51, Loss: 0.5556530242695121
Epoch: 52, Loss: 0.5561214016896955
Epoch: 53, Loss: 0.55566878280278
Epoch: 54, Loss: 0.5556357022835298
Epoch: 55, Loss: 0.5555755676206354
Epoch: 56, Loss: 0.5554641690230772
Epoch: 57, Loss: 0.5555509729610847
Epoch: 58, Loss: 0.5559595439792349
Epoch: 59, Loss: 0.554919367233354
Epoch: 60, Loss: 0.5557441146493182
Epoch: 61, Loss: 0.5560518335285928
Epoch: 62, Loss: 0.55583821348706

In [102]:
# Deploy the trained model
from sagemaker.pytorch import PyTorchModel
from sklearn.metrics import roc_auc_score

model = PyTorchModel(
    entry_point="predict.py",
    role=role, 
    framework_version="1.0",
    model_data=estimator.model_data,
    source_dir="pytorch"
)

# deploy your model to create a predictor
predictor = model.deploy(initial_instance_count=1, instance_type="ml.t2.medium")

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


---------------!

In [103]:
# evaluating the model
# read in test data, assuming it is stored locally
test_data = pd.read_csv("data/test_full.csv", header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]
test_y_preds = np.hstack(
    predictor.predict(test_x.iloc[i: i+1000]).reshape(1, -1).squeeze() 
    for i in range(0, len(test_x), 1000))

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:10: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


In [104]:
# calculate the test roc_auc_score
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, test_y_preds)

0.6665465522617937

In [105]:
from sklearn.metrics import f1_score
f1_score(test_y, test_y_preds)

0.6716461751500147

In [106]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, test_y_preds)

0.6580580318126201

In [107]:
predictor.delete_endpoint()

## 180,200, 0.15

In [35]:
s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='csv')

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


In [37]:
# import a PyTorch wrapper
from sagemaker.pytorch import PyTorch

# specify an output path
output_path = f"s3://{bucket}/{prefix}-5-hidden-layer"

# instantiate a pytorch estimator
estimator = PyTorch(
    entry_point="train.py",
    source_dir="pytorch",
    role=role,
    framework_version="1.0",
    train_instance_count=1,
    train_instance_type="ml.c4.xlarge",
    output_path=output_path,
    sagemaker_session=sagemaker_session,
    hyperparameters={
        "input_features": 19,
        "hidden_dim": 180, 
        "output_dim": 1,
        "epochs": 200
    })
estimator.fit({'train': s3_input_train})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-08-03 01:12:51 Starting - Starting the training job...
2020-08-03 01:12:54 Starting - Launching requested ML instances.........
2020-08-03 01:14:23 Starting - Preparing the instances for training...
2020-08-03 01:15:16 Downloading - Downloading input data...
2020-08-03 01:15:51 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-08-03 01:15:52,190 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-08-03 01:15:52,193 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-08-03 01:15:52,205 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-08-03 01:15:53,617 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-08-03 01:15:53,860 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setu

Epoch: 36, Loss: 0.5537568517531571
Epoch: 37, Loss: 0.5529613730919718
Epoch: 38, Loss: 0.5527754384117404
Epoch: 39, Loss: 0.5531955315937264
Epoch: 40, Loss: 0.5534251313028711
Epoch: 41, Loss: 0.552926881296023
Epoch: 42, Loss: 0.5538589406186498
Epoch: 43, Loss: 0.5529311877539319
Epoch: 44, Loss: 0.5524387712475289
Epoch: 45, Loss: 0.5524474005323001
Epoch: 46, Loss: 0.5531326096658403
Epoch: 47, Loss: 0.553106377826313
Epoch: 48, Loss: 0.5526186141126165
Epoch: 49, Loss: 0.5532800063769916
Epoch: 50, Loss: 0.5525659887029437
Epoch: 51, Loss: 0.55296017392242
Epoch: 52, Loss: 0.5532128064373459
Epoch: 53, Loss: 0.5528453586002191
Epoch: 54, Loss: 0.5518996891652823
Epoch: 55, Loss: 0.5524170900496205
Epoch: 56, Loss: 0.5527581418721417
Epoch: 57, Loss: 0.55204646035694
Epoch: 58, Loss: 0.552366188531288
Epoch: 59, Loss: 0.5522369187422906
Epoch: 60, Loss: 0.5522297721174773
Epoch: 61, Loss: 0.5524517824140828
Epoch: 62, Loss: 0.5524415688958954
Epoch: 63, Loss: 0.5518801104849421

In [38]:
# Deploy the trained model
from sagemaker.pytorch import PyTorchModel
from sklearn.metrics import roc_auc_score

model = PyTorchModel(
    entry_point="predict.py",
    role=role, 
    framework_version="1.0",
    model_data=estimator.model_data,
    source_dir="pytorch"
)

# deploy your model to create a predictor
predictor = model.deploy(initial_instance_count=1, instance_type="ml.t2.medium")

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


---------------!

In [39]:
# evaluating the model
# read in test data, assuming it is stored locally
test_data = pd.read_csv("data/test_full.csv", header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]
test_y_preds = np.hstack(
    predictor.predict(test_x.iloc[i: i+1000]).reshape(1, -1).squeeze() 
    for i in range(0, len(test_x), 1000))

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:10: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


In [40]:
# calculate the test roc_auc_score
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, test_y_preds)

0.6102643311052031

In [41]:
from sklearn.metrics import f1_score
f1_score(test_y, test_y_preds)

0.6479330114665057

In [42]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, test_y_preds)

0.5921167628037056

In [43]:
predictor.delete_endpoint()